# **Assignment - Genetic Algorithm for the N-Queens problem.**
**Subject:** MC906/MO416 - Introduction to Artificial Intelligence 

**Authors:**

    Victor Ferreira Ferrari         - RA 187890

## Introduction

This assignment consists of a genetic algorithm based solution for the N-Queens problem.
This notebook uses the _NumPy_ library.

In [1]:
import numpy as np

The parameters for the problem and the method are below:

In [2]:
N = 8
POP_SIZE = 4
MUT_RATE = 0.5
MAX_ITER = 15000

The regular execution of this assignment is with N=8, so the original 8-Queens problem.

The population in each generation has 4 states, and every new generation replaces the last one. Mutation rate and max iterations are parameters added for this implementation, and their values were decided empirically.

The overall best solution is always saved, so that if the iteration count expires, we still have the best solution.

## Code
The entire code can be found below:

In [3]:
def nqueens_genetic():
    pop = gen_initial_population(POP_SIZE)
    
    # Fitness
    fit = []
    for i in pop:
        fit.append(fitness(i))
    fit = np.array(fit)
    
    # Best
    best_idx = np.argmax(fit)
    best_sol = pop[best_idx]
    best_val = fit[best_idx]
    
    # Stop criteria
    sol_val = np.arange(N).sum()
    iterations = 0
    
    while best_val < sol_val and iterations < MAX_ITER:
        
        # Roulette selection
        sel = selection(pop, fit, POP_SIZE)
    
        # Reproduction
        for i in range(0, len(sel), 2):
            pop[i], pop[i+1] = reproduction(sel[i], sel[i+1])
    
        # Mutation
        mutate_pop(pop, MUT_RATE)
        
        # Fitness
        for i in range(POP_SIZE):
            fit[i] = fitness(pop[i])
                
        # Update Max
        best_idx = np.argmax(fit)
        if fit[best_idx] > best_val:
            best_sol = pop[best_idx]
            best_val = fit[best_idx]
        
        iterations += 1
    
    # Board
    board = create_board(best_sol)
    print(board)
    
    print('Solution: ', best_sol)
    print('Iterations: ', iterations)
    print('Solution Value: ', best_val)
    

def gen_initial_population(amount):
    pop = []
    
    # Creating random states.
    for i in range(amount):
        code = np.random.randint(0, high=N, size=N)
        pop.append(code)
    
    return pop

def fitness(state):
        
    # Max value is the sum of elements in the range 0-N.
    value = np.arange(N).sum()
    
    # Check line.
    for j in range(len(state)):
        for i in state[j+1:]:
            value -= 1 if state[j]==i else 0
    
    # Check diagonals
    for j in range(len(state)):
        for i in range(j+1,len(state)):
            dx = state[i] - state[j]
            dy = i - j
            value -= 1 if dx == dy or dx == -dy else 0
    
    return value

def create_board(state):
    board = np.zeros((N,N))
    
    for i in range(len(state)):
        board[state[i],i] = 1
    
    return board

def selection(pop, fit, amount):
    
    # Roulette selection
    sel_idxs = np.random.choice(np.arange(amount), (amount), replace=True, p=fit/fit.sum())
    
    selected = []
    for idx in sel_idxs:
        selected.append(pop[idx])
    
    return selected    

def reproduction(x, y):
    # Single-point crossover: generate point
    point = np.random.randint(0, x.size-1)
    
    # Getting children
    c1 = np.concatenate([x[:point], y[point:]])
    c2 = np.concatenate([y[:point], x[point:]])
    
    return c1,c2

def mutate_pop(pop, rate):
    
    # Select states to mutate.
    mut_amount = int(rate*len(pop))
    mut_idx = np.random.randint(0, len(pop)-1, mut_amount) 
    size = pop[0].size
    
    # Change element to random.
    for i in mut_idx:
        change = np.random.randint(0, size, 2)
        pop[i][change[0]] = change[1]

## Step-By-Step

The initial population is generated randomly, without any restrictions outside of coding. A state is an array of N positions, representing the position of the queen in each column.

In [4]:
pop = gen_initial_population(POP_SIZE)
print(pop)

[array([6, 6, 4, 3, 2, 2, 6, 0]), array([0, 0, 0, 2, 5, 3, 2, 4]), array([4, 4, 6, 2, 7, 0, 4, 1]), array([6, 6, 4, 7, 7, 7, 2, 5])]


The fitness function returns the number of queens that are not attacking each other. That means that for each combination of pairs of queens, if they are not attacking each other, the value goes up by 1. 

The maximum value is N-1+N-2+...+1. For 8 queens, this value is 28.

In [5]:
# Fitness
fit = []
for i in pop:
    fit.append(fitness(i))
fit = np.array(fit)
print(fit)

[14 21 19 21]


To breed a new generation, first we have to **select** the individuals to breed. This is done by **roulette selection**.


In [6]:
# Roulette selection
sel = selection(pop, fit, POP_SIZE)
print(sel)

[array([6, 6, 4, 7, 7, 7, 2, 5]), array([0, 0, 0, 2, 5, 3, 2, 4]), array([4, 4, 6, 2, 7, 0, 4, 1]), array([6, 6, 4, 3, 2, 2, 6, 0])]


Then, reproduction is done through single-point crossover in pairs.

In [7]:
# Reproduction
for i in range(0, len(sel), 2):
    pop[i], pop[i+1] = reproduction(sel[i], sel[i+1])
print(pop)

[array([6, 0, 0, 2, 5, 3, 2, 4]), array([0, 6, 4, 7, 7, 7, 2, 5]), array([4, 4, 6, 3, 2, 2, 6, 0]), array([6, 6, 4, 2, 7, 0, 4, 1])]


Mutation is done by randomly altering one of the queen positions in the states. The individuals are chosen randomly, with the amount being restricted by the mutation rate.

In [8]:
# Mutation
mutate_pop(pop, MUT_RATE)
print(pop)

[array([6, 0, 0, 2, 5, 3, 2, 4]), array([0, 6, 7, 7, 7, 7, 5, 5]), array([4, 4, 6, 3, 2, 2, 6, 0]), array([6, 6, 4, 2, 7, 0, 4, 1])]


Finally, we have a new generation, and their fitness values can be calculated.

In [9]:
# Fitness
for i in range(POP_SIZE):
    fit[i] = fitness(pop[i])
print(fit)

[23 18 21 24]


We can visualize a state/solution by creating a board.

In [10]:
# Board
board = create_board(pop[0])
print(board)

[[0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


## Solution

By running the `nqueens_genetic` function, you can solve an instance of the N-Queens problem. Feel free to change the parameters and see the difference! This method is not guaranteed to return the solution, though, so an iteration limit was set.

In [11]:
N=8
POP_SIZE = 4
MUT_RATE = 0.5
MAX_ITER = 15000

nqueens_genetic()

[[0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
Solution:  [3 5 0 2 6 6 1 7]
Iterations:  15000
Solution Value:  27
